In [19]:
import os
import numpy as np
from datetime import datetime 
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint 
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import utils
from sklearn import metrics 


In [8]:

# retrieve the preprocessed data from previous notebook

%store -r X_train 
%store -r X_test 
%store -r y_train 
%store -r y_test 
%store -r y_cat 
%store -r le

In [23]:
# or get that pickle
vec_type = 'mfccs'
data_path = os.path.join('pickles', 'urbansound_'+ vec_type + '.p')

with open(data_path, 'rb') as handle:
    data = pickle.load(handle)

In [24]:
X_train, X_test, y_train, y_test = data[0], data[1], data[2], data[3]

In [25]:
## num_rows as specified by number of mfccs or mels. Columns expected to be same as <max_pad_len>
num_rows = 40
num_columns = 174
num_channels = 1

X_train = X_train.reshape(X_train.shape[0], num_rows, num_columns, num_channels)
X_test = X_test.reshape(X_test.shape[0], num_rows, num_columns, num_channels)

num_labels = y_cat.shape[1]
filter_size = 2


In [26]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, input_shape=(num_rows, num_columns, num_channels), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(GlobalAveragePooling2D())

model.add(Dense(num_labels, activation='softmax'))


In [27]:
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')


In [28]:
# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(X_test, y_test, verbose=1)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 39, 173, 16)       80        
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 19, 86, 16)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 19, 86, 16)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 18, 85, 32)        2080      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 9, 42, 32)         0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 9, 42, 32)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 8, 41, 64)        